In [1]:
import os
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"

In [2]:
import pandas as pd
import numpy as np
import torch
from tqdm import tqdm
import re
import string
from torch.utils.data import DataLoader
from torch import nn
import time
from torch.utils.data.dataset import random_split

In [3]:
print(os.environ["PYTORCH_ENABLE_MPS_FALLBACK"])

1


In [4]:
data_dir = "./data/train.csv"
data_csv = pd.read_csv(data_dir)
data_csv

,qid,question_text,target
0,00002165364db923c7e6,How did Quebec nationalists see their province...,0
1,000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0
2,0000412ca6e4628ce2cf,Why does velocity affect time? Does velocity a...,0
3,000042bf85aa498cd78e,How did Otto von Guericke used the Magdeburg h...,0
4,0000455dfa3e01eae3af,Can I convert montra helicon D to a mountain b...,0
...,...,...,...
1306117,ffffcc4e2331aaf1e41e,What other technical skills do you need as a c...,0
1306118,ffffd431801e5a2f4861,Does MS in ECE have good job prospects in USA ...,0
1306119,ffffd48fb36b63db010c,Is foam insulation toxic?,0
1306120,ffffec519fa37cf60c78,How can one start a research project based on ...,0


In [5]:
insincere_rows_ = data_csv[data_csv.target == 1]
sincere_rows_ = data_csv[data_csv.target == 0]

num_insincere_ = len(insincere_rows_)

balanced_data_csv = pd.concat(
    [insincere_rows_, sincere_rows_[:num_insincere_]]
).sample(frac = 1).reset_index(drop=True)

balanced_data_csv.target.value_counts()

target
1    80810
0    80810
Name: count, dtype: int64

In [6]:
train_csv = balanced_data_csv[:80_000]
test_csv = balanced_data_csv[80_000:]

In [7]:
glove_dir = "./data/embeddings/glove.840B.300d/glove.840B.300d.txt"

glove = {}

with open(glove_dir, encoding = "utf8") as f:
    for line in tqdm(f):
        values = line.split(" ")
        word = values[0]
        vector = np.asarray(values[1:], dtype = 'float32')
        glove[word] = vector

vocab_size = len(glove)
embedding_size = len(glove.get('a'))

print('Found %s word vectors.' % vocab_size)
print(f'Each with length: {embedding_size}')

0it [00:00, ?it/s]

2196017it [00:57, 38411.87it/s]

Found 2196016 word vectors.
Each with length: 300


In [8]:
embedding_matrix = np.zeros((vocab_size, embedding_size), dtype = 'float32')

for i, (word, vector) in enumerate(tqdm(glove.items())):
    embedding_matrix[i] = vector

embedding_matrix = torch.from_numpy(embedding_matrix)
embedding_matrix.size()

  0%|          | 0/2196016 [00:00<?, ?it/s]

100%|██████████| 2196016/2196016 [00:05<00:00, 398606.35it/s]


torch.Size([2196016, 300])

In [9]:
dictionary = {word : ind for ind, word in enumerate(glove.keys())}
vocab_set = set(glove.keys())

In [10]:
def text_to_list(text, vocab = vocab_set):
    text = text.replace('\n', ' ')
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = text.replace('/', ' ').split()
    return [word for word in text if word in vocab_set]

test_str = "Why can some birds fly, but others can't?"

text_to_list(test_str)

['Why', 'can', 'some', 'birds', 'fly', 'but', 'others', 'cant']

In [11]:
def tokenizer(text_list):
    return np.array([dictionary.get(word) for word in text_list], dtype = int)

tokenizer(text_to_list(test_str)) # np.array of word indices

array([ 640,   41,   85, 4244, 2981,   42,  462, 3308])

In [12]:
text_pipeline = lambda x : tokenizer(text_to_list(x))
label_pipeline = lambda x : x

In [13]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0]

    for _label, _text in batch:
        label_list.append(label_pipeline(_label))
        
        processed_text = torch.tensor(text_pipeline(_text), dtype = torch.int)
        text_list.append(processed_text)

        offsets.append(processed_text.size(0)) # num_words

    label_list = torch.tensor(label_list, dtype = torch.int8)
    offsets = torch.tensor(offsets[:-1], dtype = torch.int16).cumsum(dim = 0)
    text_list = torch.cat(text_list)

    return label_list.to(device), text_list.to(device), offsets.to(device)

In [14]:
class QuestionClassifier(nn.Module):
    
    def __init__(self, embedding_matrix, embed_dim, num_classes):
        super(QuestionClassifier, self).__init__()
        self.embedding = nn.EmbeddingBag.from_pretrained(
            embedding_matrix, freeze = True, sparse = False
            )
        self.fc = nn.Linear(embed_dim, num_classes)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)

In [15]:
num_classes = 2

model = QuestionClassifier(
    embedding_matrix, embedding_size, num_classes
    ).to(device)

In [16]:
def train(dataloader):
    model.train()
    total_acc, total_count = 0, 0
    log_interval = 500
    start_time = time.time()

    for idx, (label, text, offsets) in enumerate(dataloader):
        optimizer.zero_grad()
        predictied_label = model(text, offsets)
        loss = criterion(predictied_label, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()
        total_acc += (predictied_label.argmax(1) == label).sum().item()
        total_count += label.size(0)
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print(
                "| epoch {:3d} | {:5d}/{:5d} batches "
                "| accuracy {:8.3f}".format(
                    epoch, idx, len(dataloader), total_acc / total_count
                )
            )
            total_acc, total_count = 0, 0
            start_time = time.time()

def evaluate(dataloader):
    model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, (label, text, offsets) in enumerate(dataloader):
            predicted_label = model(text, offsets)
            loss = criterion(predicted_label, label)
            total_acc += (predicted_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
    return total_acc / total_count

In [17]:
def to_map_style_dataset(iter_data):
    r"""Convert iterable-style dataset to map-style dataset.

    args:
        iter_data: An iterator type object. Examples include Iterable datasets, string list, text io, generators etc.


    Examples:
        >>> from torchtext.datasets import IMDB
        >>> from torchtext.data import to_map_style_dataset
        >>> train_iter = IMDB(split='train')
        >>> train_dataset = to_map_style_dataset(train_iter)
        >>> file_name = '.data/EnWik9/enwik9'
        >>> data_iter = to_map_style_dataset(open(file_name,'r'))
    """

    # Inner class to convert iterable-style to map-style dataset
    class _MapStyleDataset(torch.utils.data.Dataset):
        def __init__(self, iter_data) -> None:
            # TODO Avoid list issue #1296
            self._data = list(iter_data)

        def __len__(self):
            return len(self._data)

        def __getitem__(self, idx):
            return self._data[idx]

    return _MapStyleDataset(iter_data)

In [18]:
# Hyperparameters
EPOCHS = 10
LR = 5
BATCH_SIZE = 64

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma = 0.1)
total_accu = None
train_iter = ((row['target'], row['question_text'])
              for _, row in train_csv.drop(columns = ["qid"]).iterrows())
test_iter = ((row['target'], row['question_text'])
             for _, row in test_csv.drop(columns = ["qid"]).iterrows())
train_dataset = to_map_style_dataset(train_iter)
test_dataset = to_map_style_dataset(test_iter)
num_train = int(len(train_dataset) * 0.95)
split_train_, split_valid_ = random_split(
    train_dataset, [num_train, len(train_dataset) - num_train]
)

train_dataloader = DataLoader(
    split_train_, batch_size = BATCH_SIZE, 
    shuffle = True, collate_fn = collate_batch
)
valid_dataloader = DataLoader(
    split_valid_, batch_size = BATCH_SIZE, 
    shuffle = True, collate_fn = collate_batch
)
test_dataloader = DataLoader(
    test_dataset, batch_size = BATCH_SIZE, 
    shuffle = True, collate_fn = collate_batch
)

for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()
    train(train_dataloader)
    accu_val = evaluate(valid_dataloader)
    if total_accu is not None and total_accu > accu_val:
        scheduler.step()
    else:
        total_accu = accu_val
    print("-" * 59)
    print(
        "| end of epoch {:3d} | time: {:5.2f}s | "
        "valid accuracy {:8.3f} ".format(
            epoch, time.time() - epoch_start_time, accu_val
        )
    )
    print("-" * 59)

/Users/andywang/anaconda3/envs/learnpytorch/lib/python3.11/site-packages/torch/nn/functional.py:2420: UserWarning: The operator 'aten::_embedding_bag_forward_only' is not currently supported on the MPS backend and will fall back to run on the CPU. This may have performance implications. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/mps/MPSFallback.mm:13.)
  ret, _, _, _ = torch.embedding_bag(


| epoch   1 |   500/ 1188 batches | accuracy    0.746
| epoch   1 |  1000/ 1188 batches | accuracy    0.762
-----------------------------------------------------------
| end of epoch   1 | time: 274.36s | valid accuracy    0.770 
-----------------------------------------------------------
| epoch   2 |   500/ 1188 batches | accuracy    0.760
| epoch   2 |  1000/ 1188 batches | accuracy    0.758
-----------------------------------------------------------
| end of epoch   2 | time: 247.82s | valid accuracy    0.755 
-----------------------------------------------------------
| epoch   3 |   500/ 1188 batches | accuracy    0.771
| epoch   3 |  1000/ 1188 batches | accuracy    0.769
-----------------------------------------------------------
| end of epoch   3 | time: 247.71s | valid accuracy    0.775 
-----------------------------------------------------------
| epoch   4 |   500/ 1188 batches | accuracy    0.774
| epoch   4 |  1000/ 1188 batches | accuracy    0.769
----------------------

In [19]:
print("checking the results of test dataset.")
accu_test = evaluate(test_dataloader)
print("test accuracy {:8.3f}".format(accu_test))

checking the results of test dataset.
test accuracy    0.768


In [37]:
quora_label = {0 : "Sincere", 1 : "Insincere"}

def predict(text, text_pipeline):
    with torch.no_grad():
        text = torch.tensor(text_pipeline(text), dtype = torch.int)
        output = model(text, torch.tensor([0]))
        return output.argmax(1).item()

example = "can a black person be racist?"

print("This question is %s." % quora_label[predict(example, text_pipeline)])

This question is Insincere.
